# Script for å legge til v440 klassifikasjon i IFC

NS8360 definerer hvordan man skal legge til klassifikasjon i IFC filer. 

Dette skriptet er laget for å forberede grunnlag for programvarer som deltar i [openLAB : Interoperate] med programmer som mottar åpne filformater klassifisert med v440. 

Dette eksempelet tar inn en IFC modell og lager v440 klassifikasjoner. def get_v440_uris(label) funksjonen er brukt til å gjøre oppslag i v440 og Bjørnar Markussen har bidratt med å lage IFC modellen og til å velge ut riktig klasse for formålet. 

Scriptet kan gjenbrukes etter eget ønske og følger lisens av repo (og anvendte biblioteker). 

In [1]:
import ifcopenshell as ios 
import requests 
import uuid

In [2]:
file = ios.open("../modellfiler/Storbruen.ifc")

In [3]:
products=file.by_type("IfcProduct")
len(products)

908

In [4]:
# Helper function to get the material of an IfcObject
def get_material(ifcObject):
    assert ifcObject.is_a("IfcObject")
    if ifcObject.HasAssociations != ():
        material = ifcObject.HasAssociations[0].RelatingMaterial.Name
    else:
        material = None
    return material

In [5]:
## function to take an IfcProduct object and print out some info about it
# ref.: https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifckernel/lexical/ifcproduct.htm
def print_IfcObject(ifcObject): 
    assert ifcObject.is_a("IfcObject")
    # type
    ifcType = ifcObject.is_a()
    print(f"Ifc type:\t {ifcType}")
    # root attributes
    guid = ifcObject.GlobalId
    print(f"Guid:\t\t {guid}")
    name = ifcObject.Name
    print(f"Name:\t\t {name}")
    description = ifcObject.Description
    print(f"Description:\t {description}")
    
    # Material 
    material = get_material(ifcObject)
    print(f"Material:\t {material}")
    
    # Object attributes
    object_type = ifcObject.ObjectType
    
    #Tag attributes 
    if ifcObject.is_a("IfcElement"):
        tag = ifcObject.Tag
    else:
        tag = None
    print(f"Tag:\t {tag}")
    
    print(f"Type:\t\t {object_type}\n")
    #return ifcType,guid,name,description,material,object_type

In [6]:
# materialtyper brukt 
material_types = set([get_material(product) for product in products])
material_types

{'CONCRETE/B45', None, 'STEEL/S355NL'}

In [7]:
#produkt typer og beskrivelse av hvert produkt
product_types = set([product.is_a() for product in products])
print(product_types)
for product in products[:15]: # remove [:15] to print the whole list of products
    print_IfcObject(product)


{'IfcSlab', 'IfcBuildingStorey', 'IfcBuilding', 'IfcWall', 'IfcColumn', 'IfcSite', 'IfcPlate', 'IfcPile', 'IfcFooting'}
Ifc type:	 IfcSite
Guid:		 0d7Fg7GZj2MxRJj7fJ8JTP
Name:		 Markusplassen
Description:	 None
Material:	 None
Tag:	 None
Type:		 None

Ifc type:	 IfcBuilding
Guid:		 3hly7pMOTAjgSPAEkiYxWH
Name:		 Storbruen
Description:	 None
Material:	 None
Tag:	 None
Type:		 None

Ifc type:	 IfcBuildingStorey
Guid:		 0cBzSKq19B2fg9breFcxZK
Name:		 Bru
Description:	 None
Material:	 None
Tag:	 None
Type:		 None

Ifc type:	 IfcWall
Guid:		 2$g5PKLDbDcQ_QHQTROYUh
Name:		 Landkar
Description:	 OtHLGn-WTEah7oWDYBRV_g_10
Material:	 CONCRETE/B45
Tag:	 IDcb67dd68-d789-40d7-96cb-8e243a59cbca
Type:		 OtHLGn-WTEah7oWDYBRV_g_10

Ifc type:	 IfcWall
Guid:		 1eU5UnQ5f66wyi_$sXLJFy
Name:		 Landkar
Description:	 OtHLGn-WTEah7oWDYBRV_g_13
Material:	 CONCRETE/B45
Tag:	 IDd22590c1-2cb1-49a8-85d5-4ef82864f831
Type:		 OtHLGn-WTEah7oWDYBRV_g_13

Ifc type:	 IfcWall
Guid:		 2tXynej51CR904PE0qycS5
Name:		 Landka

In [8]:
# Produkt navn og Ifc typer
unique_product_names = set([(product.Name,product.is_a()) for product in products])
print(unique_product_names)

{('endeskjørt', 'IfcSlab'), ('Landkar', 'IfcWall'), ('Landkar', 'IfcFooting'), ('Underbygning', 'IfcBuildingStorey'), ('Fundament', 'IfcFooting'), ('Pel', 'IfcPile'), ('Markusplassen', 'IfcSite'), ('Overbygning', 'IfcBuildingStorey'), ('søyle', 'IfcColumn'), ('Bru', 'IfcBuildingStorey'), ('Storbruen', 'IfcBuilding'), ('brubjelke', 'IfcPlate')}


In [9]:
## Ordbok for å knytte IfcRoot.Name og v440 klassifisering 
# TODO: legg til kode... 
v440_obj_dict = {
    
    "Underbygning":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Underbygning",
                    "label":"Underbygning",
                    "code":"C"},
    "Pel":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Grunnen_Peler",
           "label":"Peler",
            "code":"B2"},
    "Fundament":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Pilar_Fundament",
                 "label":"Fundament",
                 "code":"C21"},
    "søyle":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Pilar_Soyle_skive",
             "label":"Soyle/skive",
             "code":"C23"},
    "Storbruen":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Bjelkebru_platebaerere_konstant_hoyde",
                 "label":"Bjelkebru platebaerere konstant hoyde sveiset med sveiseskjoter uten samvirke",
                 "code":"37"},
    "Landkar":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Landkar",
               "label":"Landkar",
               "code":"C1"},
    "Markusplassen":{"uri":"http://markusplassen.no/v440/",
                     "label":"Nordlenningen",
                     "code":":-D"},
    "Bru":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Brudekke_Slitelag_Brudekke_sek_baeresystem",
           "label":"Bru i fylling",
           "code":"E1"},
    "endeskjørt":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Buekonstruksjon_Langsgaende_bjelke",
                  "label":"Langsgaende bjelke",
                  "code":"D45"},
    "Overbygning":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Overbygning",
                   "label":"Overbygning",
                   "code":"D"},
    "brubjelke":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Buekonstruksjon_Langsgaende_bjelke",
                 "label":"Langsgaende bjelke",
                 "code":"D45"}
}

In [10]:
v440_material_dict = {
    "CONCRETE/B45":{"uri":"http://rdf.vegdata.no/V440/v440-owl#Konstruksjonsmaterialer_Materialtyper_Betong",
                   "label":"Betong, plasstopt",
                    "code":"1"},
    'STEEL/S355NL':{"uri":"http://rdf.vegdata.no/V440/v440-owl#Utstyr_Fugetyper_Stalplatefuge_slepeplate",
                   "label":"Stalplatefuge, slepeplate, fjaerbelastet",
                    "code":"51"
                   }
}

In [11]:
# Hjelpe funksjon for å søke i v440 via sparql
def get_v440_uris(label):
    url = "http://rdfspatial.vegdata.no:7200/repositories/V440"
    query ="""SELECT ?uri ?label ?code
                WHERE {
                ?uri rdfs:label ?label .
                FILTER(STRSTARTS(?label,"%s"))
                }"""%label
    r = requests.get(url, params = {"Accept": "application/json", 'query': query})
    return r.json()["results"]["bindings"]
get_v440_uris("Underbygning")

[{'label': {'xml:lang': 'no', 'type': 'literal', 'value': 'Underbygning'},
  'uri': {'type': 'uri',
   'value': 'http://rdf.vegdata.no/v440-owl#Underbygning'}},
 {'label': {'xml:lang': 'no', 'type': 'literal', 'value': 'Underbygning'},
  'uri': {'type': 'uri',
   'value': 'http://rdf.vegdata.no/V440/v440-owl#Underbygning'}}]

In [12]:
## Helper functions to classify IFC models with v440 (or other classifications)
# ref. NS8360 -- classifying IFC models 

# function for creating ifc_guid
create_guid = lambda: ios.guid.compress(uuid.uuid1().hex)

v440baseUrl = 'http://rdf.vegdata.no/V440/v440-owl'
# create v440 classification
# Is used to represent the v440 library in an Ifc file. There should be one and only one instance with Name “v440”.
# docs: https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcexternalreferenceresource/lexical/ifcclassification.htm
v440_classification = file.createIfcClassification(v440baseUrl,'nb-NO',None,'Håndbok V440')

# create v440 classification reference in ifc according to NS8360 level 2 
# The classification reference is where you store the actual classification. 
# There should be one and only one classification reference for each classification used.
# docs: https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcexternalreferenceresource/lexical/ifcclassificationreference.htm
def create_v440_classification_reference(uri,label,code):
    print(uri,code,label,v440_classification)
    return file.createIfcClassificationReference(uri, code, label, v440_classification)

# Create the IfcRelAccociatesClassification based on an NS3451 classification reference
# Used to create the actual relationship between the classification item and the objects being classified. 
# docs: https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifckernel/lexical/ifcrelassociatesclassification.htm
def classify_ifc_with_v440(v440_dict_obj):
    objects = v440_dict_obj["objects"]
    uri = v440_dict_obj["uri"]
    label = v440_dict_obj["label"]
    code = v440_dict_obj["code"]
    owner_history = objects[0].OwnerHistory
    name = 'Handbok v440'
    classification = create_v440_classification_reference(uri,label,code)

    return file.createIfcRelAssociatesClassification(create_guid(),owner_history,name,None,objects,classification)
    

In [13]:
## Legge objecter til i v440 obj dict

for product in products:
    if "objects" in v440_obj_dict[product.Name]:
        v440_obj_dict[product.Name]["objects"].append(product)
    else:
        v440_obj_dict[product.Name]["objects"] = [product]
    

In [14]:
## legger til materialtyper til v440 material dict
for product in products:
    material = get_material(product)
    if material != None:
        if "objects" in v440_material_dict[material]:
            v440_material_dict[material]["objects"].append(product)
        else:
            v440_material_dict[material]["objects"] = [product]

In [15]:
# sjekke v440_obj ordbokens innhold
for key in v440_obj_dict:
    print(v440_obj_dict[key]["uri"],v440_obj_dict[key]["label"],len(v440_obj_dict[key]["objects"]))

http://rdf.vegdata.no/V440/v440-owl#Underbygning Underbygning 1
http://rdf.vegdata.no/V440/v440-owl#Grunnen_Peler Peler 126
http://rdf.vegdata.no/V440/v440-owl#Pilar_Fundament Fundament 7
http://rdf.vegdata.no/V440/v440-owl#Pilar_Soyle_skive Soyle/skive 7
http://rdf.vegdata.no/V440/v440-owl#Bjelkebru_platebaerere_konstant_hoyde Bjelkebru platebaerere konstant hoyde sveiset med sveiseskjoter uten samvirke 1
http://rdf.vegdata.no/V440/v440-owl#Landkar Landkar 6
http://markusplassen.no/v440/ Nordlenningen 1
http://rdf.vegdata.no/V440/v440-owl#Brudekke_Slitelag_Brudekke_sek_baeresystem Bru i fylling 1
http://rdf.vegdata.no/V440/v440-owl#Buekonstruksjon_Langsgaende_bjelke Langsgaende bjelke 5
http://rdf.vegdata.no/V440/v440-owl#Overbygning Overbygning 1
http://rdf.vegdata.no/V440/v440-owl#Buekonstruksjon_Langsgaende_bjelke Langsgaende bjelke 752


In [16]:
# sjekke v440_material ordbokens innhold
for key in v440_material_dict:
    print(v440_material_dict[key]["uri"],v440_material_dict[key]["label"],len(v440_material_dict[key]["objects"]))

http://rdf.vegdata.no/V440/v440-owl#Konstruksjonsmaterialer_Materialtyper_Betong Betong, plasstopt 25
http://rdf.vegdata.no/V440/v440-owl#Utstyr_Fugetyper_Stalplatefuge_slepeplate Stalplatefuge, slepeplate, fjaerbelastet 878


In [17]:
v440_obj_dict.keys()

dict_keys(['Underbygning', 'Pel', 'Fundament', 'søyle', 'Storbruen', 'Landkar', 'Markusplassen', 'Bru', 'endeskjørt', 'Overbygning', 'brubjelke'])

In [18]:
# Klassifisere objekter i IFC modellen med riktig v440 type
for key in v440_obj_dict:
    print(key)
    classify_ifc_with_v440(v440_obj_dict[key])

Underbygning
http://rdf.vegdata.no/V440/v440-owl#Underbygning C Underbygning #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
Pel
http://rdf.vegdata.no/V440/v440-owl#Grunnen_Peler B2 Peler #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
Fundament
http://rdf.vegdata.no/V440/v440-owl#Pilar_Fundament C21 Fundament #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
søyle
http://rdf.vegdata.no/V440/v440-owl#Pilar_Soyle_skive C23 Soyle/skive #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
Storbruen
http://rdf.vegdata.no/V440/v440-owl#Bjelkebru_platebaerere_konstant_hoyde 37 Bjelkebru platebaerere konstant hoyde sveiset med sveiseskjoter uten samvirke #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
Landkar
http://rdf.vegdata.no/V440/v440-owl#Landkar C1 Landkar #319931=IfcClassification('htt

In [19]:
v440_material_dict.keys()

dict_keys(['CONCRETE/B45', 'STEEL/S355NL'])

In [20]:
# klassifisere IFC modell med riktig v440 material type
for key in v440_material_dict:
    print(key)
    classify_ifc_with_v440(v440_material_dict[key])

CONCRETE/B45
http://rdf.vegdata.no/V440/v440-owl#Konstruksjonsmaterialer_Materialtyper_Betong 1 Betong, plasstopt #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')
STEEL/S355NL
http://rdf.vegdata.no/V440/v440-owl#Utstyr_Fugetyper_Stalplatefuge_slepeplate 51 Stalplatefuge, slepeplate, fjaerbelastet #319931=IfcClassification('http://rdf.vegdata.no/V440/v440-owl','nb-NO',$,'Håndbok V440')


In [21]:
len(file.by_type("IfcRelAssociatesClassification"))

13

In [22]:
## Lagre endringer i ny fil
file.write("../modellfiler/Storbruenv440_klassifisert.ifc")

In [23]:
file2 = ios.open("../modellfiler/Storbruenv440_klassifisert.ifc")

In [24]:
products2 = file2.by_type("IfcProduct")

In [25]:
products2[10].HasAssociations[2].RelatingClassification

#319956=IfcClassificationReference('http://rdf.vegdata.no/V440/v440-owl#Utstyr_Fugetyper_Stalplatefuge_slepeplate','51','Stalplatefuge, slepeplate, fjaerbelastet',#319931)